## Prática de Q-learning

#### O que vamos ver?
   Problema do Taxi Inteligente

<img src="101/imgs/taxi.png" alt="Drawing" style="width: 300px;"/>

Ambiente disponível no toolkit de desenvolimento [Gym](https://gym.openai.com/)

In [1]:
## Instalação biblioteca
#!pip install cmake 'gym[atari]' scipy

### Ambiente do problema
Segundo a documentação:
- "There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions."

In [3]:
from IPython.display import clear_output
from time import sleep
import numpy as np
import gym
import random
env = gym.make("Taxi-v3").env

#mostra o ambiente
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset() # reseta ambiente e retorna um estado inicial randomicamente
env.render()

print("Ações possíveis = {}".format(env.action_space))
print("Estados possíveis = {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Ações possíveis = Discrete(6)
Estados possíveis = Discrete(500)


In [5]:
# (linha do taxi, coluna do taxi, índice do passageiro, índice de destino)
state = env.encode(3, 1, 2, 0) 
print("Estado:", state)

#fixar o estado
env.s = state
env.render()

Estado: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



#### Matriz de transição de estados x ações

In [6]:
#verificar os valores de recompensa associados ao estado 328
#{action: [(probability, nextstate, reward, done)]}
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

### Treinar agente na força bruta

In [7]:
env.s = 328  # fixar ambiente para ilustrar

#inicializar as variáveis
epochs = 0
penalties, reward = 0, 0
frames = [] 

done = False

while not done:
    #seleciona ação randomicamente
    action = env.action_space.sample()
    #escolhe ação e ambiente retorna feedback
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Atribuir dados para mostrar visualização
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Número de épocas: {}".format(epochs))
print("Número de penalidades: {}".format(penalties))

Número de épocas: 368
Número de penalidades: 116


#### Visualizar como o agente vai escolhendo 

In [9]:


def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 368
State: 0
Action: 5
Reward: 20


### Usar q-learning para treinamento do agente

In [10]:
#Passo 1. - slide 6
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [11]:
total_episodes = 100001        
total_test_episodes = 100     
max_steps = 99                

learning_rate = 0.7           
gamma = 0.618                 

epsilon = 1.0                 
max_epsilon = 1.0             
min_epsilon = 0.01            
decay_rate = 0.01             

In [12]:
for episode in range(total_episodes):
    # Reseta to ambiente
    state = env.reset()
    step = 0
    done = False
    
    for step in range(max_steps):
        # Passo 2. Escolher uma ação do estado atual
        ## obter número randômico entre 0 e 1
        exp_exp_tradeoff = random.uniform(0,1)
        
        ## Se este número > epsilon --> então faz aproveitamento, escolhendo o maior q-valor deste estado
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(q_table[state,:])
        
        # Escolha aleatória --> exploração
        else:
            action = env.action_space.sample()
        
        # Passo 3. Toma a ação (a) e observa o estado de saída (s') e a recompensa (r)
        new_state, reward, done, info = env.step(action)

        # Passo 4. Atualização Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        q_table[state, action] = q_table[state, action] + learning_rate * (reward + gamma * 
                                    np.max(q_table[new_state, :]) - q_table[state, action])
                
        # O novo estado passa a ser
        state = new_state
       
        # If done : finish episode
        if done == True: 
            break
            
        if reward == -10:
            penalties += 1
        
        if episode % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {episode}")
    
    
    # Reduzir o epsilon para termos menos exploração e mais aproveitamento
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

Episode: 100000


In [13]:
penalties

7004

In [16]:
### Teste q-learning
env.reset()
rewards = []
frames = [] 

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):
        # Escolhe ação que tem o valor máximo de recompensa futura esperada
        action = np.argmax(q_table[state,:])
        
        new_state, reward, done, info = env.step(action)

        frames.append({
        'frame': env.render(mode='ansi'),
        'state': action,
        'action': action,
        'reward': reward
        })

        
        total_rewards += reward
        
        if reward == -10:
            penalties += 1
            
        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score total médio: " +  str(sum(rewards)/total_test_episodes))


Score total médio: 7.84


In [17]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1316
State: 5
Action: 5
Reward: 20


In [13]:
## Teste ações randômicas
env.reset()
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):
        #seleciona ação randomicamente
        action = env.action_space.sample()
        #escolhe ação e ambiente retorna feedback
        state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if reward == -10:
            penalties += 1
            
        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score total médio: " +  str(sum(rewards)/total_test_episodes))

Score total médio: -8.29


In [15]:
penalties

7004